In [1]:
!kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d mathchi/diabetes-data-set

Dataset URL: https://www.kaggle.com/datasets/mathchi/diabetes-data-set
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 14.5MB/s]


In [4]:
## Extract  the Dataset

import zipfile

with zipfile.ZipFile('diabetes-data-set.zip', 'r') as zip_ref:
    zip_ref.extractall()
    print('Extracted')

Extracted


In [5]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn import svm


## **Data Collection And Analysis**

### **PIMA Diabetetes DataSet**

In [6]:
diabetes_df = pd.read_csv('/content/diabetes.csv')

In [7]:
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
## Dimesnion of the Dataset
diabetes_df.shape

(768, 9)

In [9]:
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [10]:
## Getting the Stastical Mesuresof the Data

diabetes_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [11]:
diabetes_df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

**0 --> Non - Diabitic**

**1--> Diabetic**

In [12]:
diabetes_df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [13]:
## Seperating the Data And Lables
X = diabetes_df.drop(columns = 'Outcome',axis= 1)

y = diabetes_df['Outcome']

In [14]:
pd.DataFrame(X)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [15]:
pd.DataFrame(y)

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


## **Data Standardization**

In [16]:
scaler = StandardScaler()


In [17]:
scaler.fit(X)

StandardScaler()

In [18]:
standardized_data = scaler.transform(X)

In [19]:
print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [20]:
X = standardized_data
y = diabetes_df['Outcome']

In [21]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

#Train -Test- Split

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state = 42)

In [23]:
print("X -> shape",X.shape)
print("X_Train ->",X_train.shape," X_test->",X_test.shape)
print("y -> shape",y.shape)
print("y_Train ->",y_train.shape," y_test->",y_test.shape)

X -> shape (768, 8)
X_Train -> (614, 8)  X_test-> (154, 8)
y -> shape (768,)
y_Train -> (614,)  y_test-> (154,)


### **Train the Model**

In [24]:
classifier = svm.SVC(kernel = 'linear')


In [25]:
 # Traing the support vector machine Classifier
classifier.fit(X_train,y_train)

SVC(kernel='linear')

### **Model Evlauation**

In [26]:
#Accurarcy Score

#Accuracy score on the training data

X_train_prediction = classifier.predict(X_train)

training_data_accuracy = accuracy_score(X_train_prediction,y_train)


In [27]:
print("Accuracy Score of the Training Data:-",training_data_accuracy)



Accuracy Score of the Training Data:- 0.7915309446254072


In [28]:
##Accuracy Score on the test data

X_test_prediction = classifier.predict(X_test)

test_data_accuracy = accuracy_score(X_test_prediction,y_test)


In [29]:
print("Accuracy Score of the Test Data:-",test_data_accuracy)

Accuracy Score of the Test Data:- 0.7207792207792207


### **Making a Predictive Sysytem**

In [30]:
input_data = (4,110,92,0,0,37.6,0.191,30)


# changeing the input data to numpy array

input_data_as_numpy_array = np.asarray(input_data)



In [31]:
input_data_as_numpy_array.shape

(8,)

In [32]:
# reshape the numpy array as we are predicting for one instance

#there are 760 data point but here, we are doing one datapoint predict so , we reshape as (1,-1)
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

In [33]:
input_data_as_numpy_array.shape

(8,)

In [34]:
'''#Standarsize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)


print('-'*50)

prediction = classifier.predict(std_data)
print(prediction)



if(prediction[0] ==0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')




print('-'*50)
'''

"#Standarsize the input data\nstd_data = scaler.transform(input_data_reshaped)\nprint(std_data)\n\n\nprint('-'*50)\n\nprediction = classifier.predict(std_data)\nprint(prediction)\n\n\n\nif(prediction[0] ==0):\n  print('The person is not diabetic')\nelse:\n  print('The person is diabetic')\n\n\n\n\nprint('-'*50)\n"

In [35]:
#


input_data = (5,166,72,19,175,25.8,0.587,51)

#Changing the input_data to numpy array

input_data_as_numpy_array = np.asarray(input_data)

#reshape the aray as we predicting for one instance

input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = classifier.predict(input_data_reshaped)

print(prediction)

if(prediction[0] ==0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic


## **Saving the Model**

In [36]:
import pickle

In [41]:

#Naming the file
filename = 'trained_model_diabetes.sav'

#saving the model in write binary format (classifier)
pickle.dump(classifier,open(filename,'wb'))


In [42]:
##Loading the Saved Model


loaded_model = pickle.load(open('trained_model_diabetes.sav','rb'))


In [43]:
#


input_data = (5,166,72,19,175,25.8,0.587,51)

#Changing the input_data to numpy array

input_data_as_numpy_array = np.asarray(input_data)

#reshape the aray as we predicting for one instance

input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = loaded_model.predict(input_data_reshaped)

print(prediction)

if(prediction[0] ==0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic
